In [1]:
# Import the necessary libraries
# TensorFlow and tf.keras
import tensorflow as tf
import keras
# Helper libraries
import numpy as np
from os import listdir
from os.path import join
import cv2
import pandas
import os
import random

/home/liyanc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
data = "/media/liyanc/零碎东西/deeplearning/all/train/train"
# List out the directories inside the main input folder
folders = os.listdir(data)
print(folders)

['cat', 'dog']


In [5]:
# Import the images and resize them to a 128*128 size
# Also generate the corresponding labels

image_names = []
train_labels = []
train_images = []

size = 64,64
i = 0
for folder in folders:
    for file in os.listdir(os.path.join(data,folder)):
        if file.endswith("jpg"):
            if i%1000 ==0:
                print(str(i))
            i += 1
            image_names.append(os.path.join(data,folder,file))
            train_labels.append(folder)
            img = cv2.imread(os.path.join(data,folder,file))
            im = cv2.resize(img,size)
            train_images.append(im)
        else:
            continue

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000


In [6]:
len(train_images)

25000

In [7]:
# Transform the image array to a numpy type
train = np.array(train_images)
train.shape

(25000, 64, 64, 3)

In [8]:
# Reduce the RGB values between 0 and 1
train = train.astype('float32') / 255.0

In [9]:
label_dummies = pandas.get_dummies(train_labels)
label_dummies.shape

(25000, 2)

In [10]:
# Extract the labels
labels =  label_dummies.values.argmax(1)
label = pandas.get_dummies(labels)
label.shape

(25000, 2)

In [11]:
# pandas.unique(train_labels)
# label

In [12]:
# # Shuffle the labels and images randomly for better results

# union_list = list(zip(train, label))
# random.shuffle(union_list)
# train,labels = zip(*union_list)

# # Convert the shuffled list to numpy array type

# train = np.array(train)
# # labels = np.array(labels)
# labels = np.array(labels)

In [12]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), input_shape=(64, 64, 3), activation='relu',padding='same'))
model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu',padding='same'))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu',padding='same'))
model.add(keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu',padding='same'))
model.add(keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2, activation='softmax'))
opt = keras.optimizers.adam(lr=1e-5)
model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
__________

In [22]:
from keras.Model import load_weights
model = load_weights('/media/liyanc/零碎东西/deeplearning/model/model0824.h5',by_name=True)

ModuleNotFoundError: No module named 'keras.Model'

In [12]:
# Compute the model parameters

# model.compile(optimizer=tf.train.AdamOptimizer(), 
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
# label_dummies.shape

In [17]:
model.fit(train,label,batch_size=16, epochs=1 , verbose=2, validation_split=0.1)

Train on 22500 samples, validate on 2500 samples
Epoch 1/1
 - 46s - loss: 0.6888 - acc: 0.5524 - val_loss: 0.7476 - val_acc: 0.0000e+00


## save model

In [15]:
model.save('/home/liyanc/model0902.h5',overwrite=True)

## accurate cal

In [21]:
from keras.preprocessing import image
result = 0

# with open('/media/liyanc/零碎东西/deeplearning/all/submission_file.csv','w') as f:
#     f.write('id,label\n')
maxNum = 5000
for i in range(1,maxNum):#12501
    
    image_path = '/media/liyanc/零碎东西/deeplearning/all/train/train/dog/dog.'+str(i)+'.jpg'
#     image_path = '/media/liyanc/零碎东西/deeplearning/all/train/train/cat/cat.'+str(i)+'.jpg'
    # 加载图像 
    img = image.load_img(image_path, target_size=(64, 64)) 
    # 图像预处理 
    x = image.img_to_array(img) 
    x = np.expand_dims(x, axis=0) 
    # 对图像进行分类 
    preds = model.predict(x)[0]
#     try:
#         result += preds[1]
#     except:
#         result = preds[1]
    if preds[1] >=0.5:
        result = result + 1
    else:
        continue
for i in range(1,maxNum):#12501
#     image_path = '/media/liyanc/零碎东西/deeplearning/all/train/train/dog/dog.'+str(i)+'.jpg'
    image_path = '/media/liyanc/零碎东西/deeplearning/all/train/train/cat/cat.'+str(i)+'.jpg'
    # 加载图像 
    img = image.load_img(image_path, target_size=(64, 64)) 
    # 图像预处理 
    x = image.img_to_array(img) 
    x = np.expand_dims(x, axis=0) 
    # 对图像进行分类 
    preds = model.predict(x)[0]
#     try:
#         result += 1-preds[1]
#     except:
#         result = 1-preds[1]
    if preds[1] <=0.5:
        result = result + 1
    else:
        continue
print(result/(maxNum*2))

0.8157


## out to csv

In [22]:
from keras.preprocessing import image
with open('/media/liyanc/零碎东西/deeplearning/all/submission_file.csv','w') as f:
    f.write('id,label\n')
    for i in range(1,12501):#12501
#         image_path = '/media/liyanc/零碎东西/deeplearning/all/train/train/dog/dog.'+str(i)+'.jpg'
        image_path = '/media/liyanc/零碎东西/deeplearning/all/test/test/'+str(i)+'.jpg'
        # 加载图像 
        img = image.load_img(image_path, target_size=(64, 64)) 
        # 图像预处理 
        x = image.img_to_array(img) 
        x = np.expand_dims(x, axis=0) 
        # 对图像进行分类 
        preds = model.predict(x)[0] 
        if preds[1] >=0.5:
            preds[1] -= 0.05
        else:
            preds[1] += 0.05
        f.write(str(i)+','+str(preds[1])+'\n')